# Preprocessing: LL10


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [3]:
# %matplotlib qt5

In [4]:
## Prep paths ##

subject = 'LL10'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [20]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL10/ieeg/LL10_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 1559659 =      0.000 ...  3046.209 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL10/ieeg/LL10_raw_clean_ieeg.fif...
    Range : 0 ... 1559659 =      0.000 ...  3046.209 secs
Ready.


## Bipolar Rereferencing

In [21]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL10_correct_channel_names.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bci_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)


In [26]:
filtered_clean_fif

<Raw | LL10_notched_filtered_clean_ieeg.fif, 149 x 1559660 (3046.2 s), ~156 kB, data not loaded>

In [23]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [24]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

RA1 RA2
RA2 RA3
RA3 RA4
RA4 RA5
RA5 RA6
RA6 RA7
RA7 RA8
RH1 RH2
RH2 RH3
RH3 RH4
RH4 RH5
RH5 RH6
RH6 RH7
RH7 RH8
RHP1 noref
RHP2 RHP3
RHP3 RHP4
RHP4 RHP5
RHP5 RHP6
RHP6 RHP7
RHP7 RHP8
RLEAS1 noref
RLEAS2 noref
RLEAS3 noref
RLEAS4 noref
RLEAS5 noref
RLEAS6 noref
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
RAC1 RAC2
RAC2 RAC3
RAC3 RAC4
RAC4 RAC5
RAC5 RAC6
RAC6 RAC7
RAC8 noref
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF10
LAC1 LAC2
LAC2 LAC3
LAC3 LAC4
LAC4 LAC5
LAC5 LAC6
LAC6 LAC7
LAC7 LAC8
LH1 LH2
LH2 LH3
LH3 LH4
LH4 LH5
LH5 LH6
LH6 LH7
LH8 noref
CH74 noref
CH75 noref
CH76 noref
CH77 noref
CH78 noref
CH79 noref
CH80 noref
CH81 noref
CH82 noref
CH83 noref
CH84 noref
CH85 noref
CH86 noref
CH87 noref
CH88 noref
CH89 noref
CH90 noref
CH91 noref
CH92 noref
CH93 noref
CH94 noref
CH95 noref
CH96 noref
CH97 noref
CH98 noref
CH99 noref
CH100 noref
CH101 noref
CH102 noref
CH103 noref
CH104 noref
CH105 noref


In [27]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 1559659  =      0.000 ...  3046.209 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=57, n_times=1559660
    Range : 0 ... 1559659 =      0.000 ...  3046.209 secs
Ready.
Added the following bipolar channels:
RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, RHP7-RHP8, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7
Writing /home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_filtered_clean_data.fif


/tmp/ipykernel_3187/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_filtered_clean_data.fif
[done]
Reading 0 ... 1559659  =      0.000 ...  3046.209 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=57, n_times=1559660
    Range : 0 ... 1559659 =      0.000 ...  3046.209 secs
Ready.
Added the following bipolar channels:
RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, RHP7-RHP8, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7
Writing /home/brooke/pacman/preprocessing/LL10

/tmp/ipykernel_3187/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_raw_clean_data.fif
[done]


In [ ]:
# Visualize it #

filtered_notch_bp_data.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

In [5]:
## Epoching the raw data ##

# load data 
bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")
events = mne.find_events(bp_raw_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_raw_fif.info['ch_names'] if x not in bp_raw_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_raw_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_raw_clean_data.fif...


/tmp/ipykernel_13751/981813422.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif")


    Range : 0 ... 1559659 =      0.000 ...  3046.209 secs
Ready.
480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 240 events and 8193 original time points ...


/tmp/ipykernel_13751/981813422.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 8193 original time points ...
Loading data for 240 events and 8193 original time points ...


## Create Other Events

In [4]:
## Load Behavioral Data ##
last_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_dot_events.csv")
last_dot_data

,trial_numeric,TrialType,sample,sample_before,event
0,2,8,96500,0,1
1,3,11,119100,0,1
2,5,10,137800,0,1
3,6,15,157400,0,1
4,8,20,194700,0,1
...,...,...,...,...,...
397,446,17,5770200,0,1
398,447,15,5782200,0,1
399,448,18,5793400,0,1
400,449,19,5811600,0,1


In [6]:
# filter to good epochs and fix trial indexing
last_dot_data['trial_numeric'] = last_dot_data['trial_numeric'] - 1

# create events
last_dot_events = last_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [9]:
# epoch the data
last_dot_epochs = mne.Epochs(bp_raw_fif, last_dot_events, 
                          event_id = 1, tmin = -.75 * 4, tmax =.75 * 3, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_dot_events.fif", overwrite = True, split_size = '1.9GB')


Not setting metadata
402 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 402 events and 10501 original time points ...


/tmp/ipykernel_2043602/260881574.py:7: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_last_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_dot_events.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 10501 original time points ...
Splitting into 2 parts
Loading data for 201 events and 10501 original time points ...
Loading data for 201 events and 10501 original time points ...


Okay, what we need to do is pul the sample of the last dot eaten on each trial

and then also the last turn around

looking for a comparison between these moments on ghost and no ghost trials

we can either get the average and then compare peaks via foof groups in hc and ofc and anterior cingulate

## Last Away

In [4]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,trial_numeric,TrialType,sample,sample_before,event
0,2,3,23270,0,1
1,3,8,27084,0,1
2,4,11,29952,0,1
3,6,16,36531,0,1
4,7,2,45158,0,1
...,...,...,...,...,...
220,235,8,846643,0,1
221,236,10,849228,0,1
222,237,15,852096,0,1
223,238,19,858496,0,1


In [5]:
# filter to good epochs and fix trial indexing
last_away_data['trial_numeric'] = last_away_data['trial_numeric'] - 1

# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [6]:
# epoch the data
last_away_epochs = mne.Epochs(bp_raw_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Not setting metadata
225 matching events found
No baseline correction applied
0 projection items activated
Loading data for 225 events and 5121 original time points ...


/tmp/ipykernel_46403/3409931828.py:7: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_raw_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


0 bad epochs dropped
Loading data for 1 events and 5121 original time points ...
Loading data for 225 events and 5121 original time points ...
